In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import json,re

In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
Fecha_Inicio = "2023-08-01"

In [5]:
""" sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
) """

' sf = Salesforce(\n    username="jmartinez@careassistance.com",\n    password="Jcmm01041983*",\n    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",\n) '

In [6]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Account",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape

(1882683, 2)

In [7]:
nuevos = "F:/Procesos Automaticos/AccuHealth/1.xlsx"
Archivo = pd.read_excel(nuevos,sheet_name='Accuhealth')

# OJO se copia rut1 a rut le saca el guoion
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP
0,25785,9512321,06060944-6,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106
1,25785,9515540,06060944-6,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109
2,25785,9519833,06060944-6,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118
3,25785,9524222,06060944-6,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113
4,25785,9525982,06060944-6,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante


In [8]:
Archivo["RUT"] = Archivo["RUT"].astype(str)
Archivo["RUT"] = Archivo["RUT"].str.replace("-", "")
Archivo["RUT"] = Archivo["RUT"].str.replace(".", "")
Archivo["RUT"] = Archivo["RUT"].str.upper()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23956\3527352582.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Archivo["RUT"] = Archivo["RUT"].str.replace(".", "")


In [9]:
Archivo.dtypes

patient_id         int64
patient_lot_id     int64
RUT               object
name              object
surname           object
date              object
severity          object
BG                object
bienestar         object
DBP               object
Medicacion        object
SBP               object
dtype: object

In [10]:
def siPrimerCero(y):
    if y[0] == "0":
        y = y[1:]
    return y

In [11]:
Archivo["RUT2"] = Archivo["RUT"].apply(lambda y: siPrimerCero(y))
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446


In [12]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23956\2920035971.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")


In [13]:
Archivo = pd.merge(
    left=Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="RUT2",
    right_on="Rut",
)
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2,Account,Rut
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446,0018c00002amPjDAAU,60609446
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446,0018c00002amPjDAAU,60609446
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446,0018c00002amPjDAAU,60609446
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446,0018c00002amPjDAAU,60609446
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446,0018c00002amPjDAAU,60609446


In [14]:
Archivo["Presión Arterial"] = np.where(Archivo["DBP"].str.len() <=4,Archivo["SBP"] + " / " + Archivo["DBP"],"0")

Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2,Account,Rut,Presión Arterial
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446,0018c00002amPjDAAU,60609446,106 / 75
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446,0018c00002amPjDAAU,60609446,109 / 70
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446,0018c00002amPjDAAU,60609446,118 / 74
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446,0018c00002amPjDAAU,60609446,113 / 79
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446,0018c00002amPjDAAU,60609446,0


In [15]:
Archivo["Dia"] = pd.to_datetime(Archivo["date"]).dt.day

Archivo["Mes"] = pd.to_datetime(Archivo["date"]).dt.month

Archivo["Mes Nombre"] = pd.to_datetime(Archivo["date"]).dt.month_name()

Archivo["Año"] = pd.to_datetime(Archivo["date"]).dt.year

Archivo["Fecha_chilena"] = pd.to_datetime(
    Archivo["date"], dayfirst=True, yearfirst=False, exact=True
)
# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
# Archivo['LocalDateTime'] = pd.to_datetime(Archivo['LocalDateTime'])
# 

Archivo["Fecha_chilena"] = Archivo["Fecha_chilena"].apply(
    lambda a: datetime.strftime(a, "%Y-%m-%d %H:%M:%S")
)

Archivo["Fecha_chilena"] = pd.to_datetime(Archivo["Fecha_chilena"])

Archivo["Fecha"] = Archivo["Fecha_chilena"].dt.strftime("%d-%m-%Y")
Archivo.to_excel("F:/Procesos Automaticos/AccuHealth/2.xlsx", index=False)


Archivo = Archivo[(Archivo["severity"] != "No realizado")]

Discriminacion fecha datos relevantes para carga

In [16]:
Archivo1 = Archivo[(Archivo["Fecha_chilena"] >= Fecha_Inicio)]
Archivo1.sort_values(by="date")

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,...,RUT2,Account,Rut,Presión Arterial,Dia,Mes,Mes Nombre,Año,Fecha_chilena,Fecha
4742,31878,9901128,146301541,Eduardo Javier,De la Rosa Rebaza,2023-08-01T00:29:24.000Z,Baja,123,Bien,NaN,...,146301541,0018c00002amTvkAAE,146301541,0,1,8,August,2023,2023-08-01 00:29:24,01-08-2023
489,29224,9901131,072492277,Guillermo Felipe,Arce Fuentes,2023-08-01T00:56:12.000Z,Baja,NaN,Bien,NaN,...,72492277,0018c00002amLvLAAU,72492277,0,1,8,August,2023,2023-08-01 00:56:12,01-08-2023
490,29224,9901132,072492277,Guillermo Felipe,Arce Fuentes,2023-08-01T00:57:33.000Z,Moderada,200,NaN,80,...,72492277,0018c00002amLvLAAU,72492277,126 / 80,1,8,August,2023,2023-08-01 00:57:33,01-08-2023
5435,32888,9901142,112571132,Danahe,Méndez Valera,2023-08-01T05:32:20.000Z,Moderada,167,Bien,NaN,...,112571132,0018c00002amSFzAAM,112571132,0,1,8,August,2023,2023-08-01 05:32:20,01-08-2023
13101,37509,9901145,120892029,Genoveva,Díaz González,2023-08-01T05:40:03.000Z,Moderada,NaN,Bien,84,...,120892029,0018c00002ao1y6AAA,120892029,116 / 84,1,8,August,2023,2023-08-01 05:40:03,01-08-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15405,39155,10193101,160836490,Erick,González Morales,2023-12-25T22:20:45.000Z,Moderada,NaN,Bien,73,...,160836490,0018c00002eUHeTAAW,160836490,111 / 73,25,12,December,2023,2023-12-25 22:20:45,25-12-2023
7307,38313,10193245,162310445,Andrés,Silva Veas,2023-12-25T23:18:38.000Z,Baja,104,Bien,NaN,...,162310445,0018c00002bKW89AAG,162310445,0,25,12,December,2023,2023-12-25 23:18:38,25-12-2023
14774,38315,10193249,131492766,Cristian,Gangas Matus,2023-12-25T23:21:21.000Z,Moderada,NaN,Bien,83,...,131492766,0018c00002ao2N2AAI,131492766,114 / 83,25,12,December,2023,2023-12-25 23:21:21,25-12-2023
14228,37821,10193251,170589882,Elizabeth,Poblete Carreño,2023-12-25T23:23:13.000Z,Alta,NaN,Bien,87,...,170589882,0018c00002cVlHSAA0,170589882,117 / 87,25,12,December,2023,2023-12-25 23:23:13,25-12-2023


In [17]:
Archivo1 = Archivo1[
    [
        "patient_lot_id",
        "Rut",
        "Fecha",
        "Fecha_chilena",
        "severity",
        "BG",
        "bienestar",
        "Medicacion",
        "Account",
        "Presión Arterial",
    ]
]
Archivo1.shape

(5030, 10)

In [18]:
results = sf.query_all(
    """
    select Id,idAccuHealth__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c 
   """
)
fichaAccu = pd.DataFrame(results["records"])

fichaAccu = fichaAccu[
    [
        "Id",
        "idAccuHealth__c",
    ]
]
fichaAccu = fichaAccu.rename(
    columns={
        "Id": "Id Account Paciente",
        "idAccuHealth__c": "Codigo",
    }
)
fichaAccu["Codigo"] = fichaAccu["Codigo"].astype(str)
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.replace("-", "")
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.replace(".", "")
fichaAccu["Codigo"] = fichaAccu["Codigo"].str.upper()

fichaAccu.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23956\2271742192.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fichaAccu["Codigo"] = fichaAccu["Codigo"].str.replace(".", "")


(8485, 2)

In [19]:
fichaAccu.head()

,Id Account Paciente,Codigo
0,a1f8c000001ovZuAAI,NONE
1,a1f8c000001ovZzAAI,NONE
2,a1f8c000001ova4AAA,NONE
3,a1f8c000001ova9AAA,NONE
4,a1f8c000001ovaEAAQ,NONE


In [20]:
fichaAccu["Codigo"] = fichaAccu["Codigo"].astype(str)


In [21]:

Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].astype(str)
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.replace("-", "")
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.replace(".", "")
Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.upper()

Archivo1["Estaono"] = Archivo1["patient_lot_id"].isin(fichaAccu["Codigo"])
Archivo1["Canal__c"] = "Chatter"
Archivo1["RecordTypeId"] = "0128c000002CYuWAAW"

Archivo1.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23956\49131534.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Archivo1["patient_lot_id"] = Archivo1["patient_lot_id"].str.replace(".", "")


,patient_lot_id,Rut,Fecha,Fecha_chilena,severity,BG,bienestar,Medicacion,Account,Presión Arterial,Estaono,Canal__c,RecordTypeId
184,9927929,112296999,16-08-2023,2023-08-16 06:18:08,Moderada,152,Bien,No,0018c00002amLvfAAE,131 / 86,True,Chatter,0128c000002CYuWAAW
185,9932225,112296999,18-08-2023,2023-08-18 06:30:46,Moderada,132,Bien,No,0018c00002amLvfAAE,118 / 75,True,Chatter,0128c000002CYuWAAW
186,9936593,112296999,21-08-2023,2023-08-21 06:22:39,Moderada,132,Bien,No,0018c00002amLvfAAE,138 / 84,True,Chatter,0128c000002CYuWAAW
187,9940895,112296999,23-08-2023,2023-08-23 06:11:17,Moderada,137,Bien,No,0018c00002amLvfAAE,126 / 78,True,Chatter,0128c000002CYuWAAW
188,9945113,112296999,25-08-2023,2023-08-25 06:21:17,Moderada,132,Bien,Sí,0018c00002amLvfAAE,123 / 77,True,Chatter,0128c000002CYuWAAW


In [22]:
Archivo1 = Archivo1.rename(
    columns={
        "patient_lot_id": "idAccuHealth__c",
        "Rut": "Rut__c",
        "Fecha_chilena": "Fecha__c",
        "BG": "Glicemia__c",
        "bienestar": "Bienestar__c",
        "Medicacion": "Medicaci_n__c",
        "Account": "Account__c",
        "Presión Arterial": "Presi_n_Arterial__c",
        "Estaono": "Estaono",
    }
)
Archivo1 = Archivo1[Archivo1["Estaono"] == False]

Archivo1 = Archivo1[
    [
        "idAccuHealth__c",
        "Fecha__c",
        "Glicemia__c",
        "Bienestar__c",
        "Medicaci_n__c",
        "Account__c",
        "Presi_n_Arterial__c",
        "Canal__c",
        "RecordTypeId",
        
    ]
]


Archivo1.to_excel("F:/Procesos Automaticos/AccuHealth/3.xlsx", index=False)